In [26]:
# Define roles and the documents they can access
class Role:
    def __init__(self, name, accessible_docs):
        self.name = name
        self.accessible_docs = accessible_docs

    def can_access(self, doc):
        return doc in self.accessible_docs

    def __str__(self):
        return f"Role: {self.name}"

# Define a group with associated roles
class Group:
    def __init__(self, name):
        self.name = name
        self.roles = []
        self.members = []

    def add_role(self, role):
        self.roles.append(role)

    def add_member(self, user):
        self.members.append(user)
        user.groups.append(self)  # Add this group to the user's list of groups

    def __str__(self):
        return f"Group: {self.name}"

# Define a user with group membership and direct roles
class User:
    def __init__(self, username):
        self.username = username
        self.groups = []
        self.direct_roles = []  # Roles assigned directly to the user

    def add_role(self, role):
        self.direct_roles.append(role)

    def can_access_document(self, document_name):
        # Check roles from groups the user belongs to
        for group in self.groups:
            for role in group.roles:
                if role.can_access(document_name):
                    return True
        # Check roles assigned directly to the user
        for role in self.direct_roles:
            if role.can_access(document_name):
                return True
        return False

    def __str__(self):
        # Print user details including groups and roles
        group_names = [group.name for group in self.groups]
        direct_role_names = [role.name for role in self.direct_roles]
        inherited_roles = {role.name for group in self.groups for role in group.roles}

        return (
            f"User: {self.username}\n"
            f"Groups: {', '.join(group_names) if group_names else 'None'}\n"
            f"Direct Roles: {', '.join(direct_role_names) if direct_role_names else 'None'}\n"
            f"Inherited Roles: {', '.join(inherited_roles) if inherited_roles else 'None'}"
        )

In [27]:
# Define the documents available in the system
DOCS = ["how to sell", "marketing techniques", "HR docs", "QA tools", "troubleshooting docs", "SRE tickets", "rule book"]

# Create roles with access to specific documents
sales_role = Role("Sales", ["how to sell", "marketing techniques"])
hr_role = Role("HR", ["HR docs", "rule book"])
qa_role = Role("QA", ["QA tools", "troubleshooting docs"])
sre_role = Role("SRE", ["SRE tickets", "troubleshooting docs"])

# Create groups and assign roles to each group
marketing_group = Group("Marketing")
marketing_group.add_role(sales_role)
marketing_group.add_role(hr_role)

tech_support_group = Group("Tech Support")
tech_support_group.add_role(qa_role)
tech_support_group.add_role(sre_role)

# Create users
user1 = User("Alice")
user2 = User("Bob")
user3 = User("Charlie")  # This user will have direct access without a group

# Add users to groups (inherit group roles)
marketing_group.add_member(user1)
tech_support_group.add_member(user2)

# Directly assign a role to a user (without adding to a group)
user3.add_role(hr_role)

# Check document access
def check_access(user, document_name):
    if user.can_access_document(document_name):
        print(f"{user.username} can access '{document_name}'")
    else:
        print(f"{user.username} cannot access '{document_name}'")

# Test access for specific documents
check_access(user1, "HR docs")            # Should print access for Alice (via group)
check_access(user2, "marketing techniques") # Should print no access for Bob
check_access(user2, "troubleshooting docs") # Should print access for Bob (via group)
check_access(user3, "HR docs")            # Should print access for Charlie (direct role)
check_access(user3, "how to sell")        # Should print no access for Charlie


Alice can access 'HR docs'
Bob cannot access 'marketing techniques'
Bob can access 'troubleshooting docs'
Charlie can access 'HR docs'
Charlie cannot access 'how to sell'
